In [270]:
import json
import sys
import re
import requests
import pandas as pd
import numpy as np
from BeautifulSoup import BeautifulSoup
import urllib2
import unidecode
import tqdm
import pickle
import pycurl
import time
import guess_language
import psycopg2
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from gensim import corpora, models, similarities
from nltk.stem.porter import PorterStemmer
from collections import defaultdict

In [80]:
def get_itunes_charts_ids(url):
    r = requests.get(url)
    c = r.content
    soup = BeautifulSoup(c)
    samples = soup.findAll("p", "buy")
    
    ids = []
    for s in samples:
        s = str(s.contents[0])
        ids.append(re.findall(r'/id([\d]+)', s)[0])
    
    return ids

In [82]:
scrape_urls = ['http://www.itunescharts.net/us/charts/podcasts/2016/01/29',
              'http://www.itunescharts.net/us/charts/podcasts/comedy/',
              'http://www.itunescharts.net/us/charts/podcasts/tv-film/',
              'http://www.itunescharts.net/us/charts/podcasts/technology/',
              'http://www.itunescharts.net/us/charts/podcasts/sport/',
              'http://www.itunescharts.net/us/charts/podcasts/games-hobbies/',
              'http://www.itunescharts.net/us/charts/podcasts/arts/',
              'http://www.itunescharts.net/us/charts/podcasts/music/',
              'http://www.itunescharts.net/us/charts/podcasts/religion-spirituality/',
              'http://www.itunescharts.net/us/charts/podcasts/science-medicine/']

In [86]:
all_ids = []
for url in scrape_urls:
    ids = get_itunes_charts_ids(url)
    all_ids = all_ids + ids

In [87]:
len(all_ids)

460

In [92]:
# add purrrcast
all_ids = all_ids + ['1041016803']

# remove duplicates
all_ids = list(set(all_ids))

In [93]:
len(all_ids)

418

In [95]:
# will store pycurl output
class Test:
   def __init__(self):
       self.contents = ''

   def body_callback(self, buf):
       self.contents = self.contents + buf
        
def run_curl(url):
    t = Test()
    c = pycurl.Curl()
    c.setopt(pycurl.URL, url)
    c.setopt(pycurl.HTTPHEADER, ['X-Apple-Store-Front: 143441-1,12', 'X-Apple-Tz: 3600'])
    c.setopt(pycurl.USERAGENT, 'iTunes/9.2.1 (Macintosh; Intel Mac OS X 10.5.8) AppleWebKit/533.16')
    c.setopt(pycurl.SSL_VERIFYHOST, 0)
    c.setopt(pycurl.SSL_VERIFYPEER, 0)
    c.setopt(pycurl.WRITEFUNCTION, t.body_callback)
    c.perform()
    return t

def clean_description(d):
    d = unidecode.unidecode(d)
    d = d.replace('\n', ' ')
    if re.findall(r'(.*) brought to you by.*', d):
       d = re.sub(r'brought to you by.*', '', d)
    if re.search(r'(.*) sponsored by.*', d):
       d = re.sub(r'sponsored by.*', '', d)
    return d

In [101]:
baseUrl = 'https://itunes.apple.com/us/podcast/id'
colNames = ['collectionId', 'podcastSummary', 'episodeNames', 'episodeDescriptions', 'alsoSubscribed']
scrapeResults = pd.DataFrame(columns=colNames)

for i in tqdm.tqdm(all_ids):
    scrapeUrl = baseUrl + str(i)
    
    # get podcast summary
    t = run_curl(scrapeUrl)
    soup = BeautifulSoup(t.contents)
    p = soup.p
    if p:
        podcastSummary = soup.p.string
    else: # redirect
        newUrl = soup.findAll(text=re.compile(r'https'))
        try:
            newUrl = newUrl[0]
            newUrl = re.sub(r'&amp;', r'&', newUrl)

            t = run_curl(newUrl)
            soup = BeautifulSoup(t.contents)
            p = soup.p
            if p:
                podcastSummary = soup.p.string
            else:
                podcastSummary = np.nan
                episodeNames = np.nan
                episodeDescriptions = np.nan
                alsoSubscribed = np.nan
                thisResult = pd.DataFrame({'collectionId' : int(i),
                                          'podcastSummary' : [podcastSummary],
                                          'episodeNames' : [episodeNames],
                                          'episodeDescriptions' : [episodeDescriptions],
                                          'alsoSubscribed' : [alsoSubscribed]})
                scrapeResults = pd.concat([scrapeResults, thisResult], axis=0)
                continue
        except:
            podcastSummary = np.nan
            episodeNames = np.nan
            episodeDescriptions = np.nan
            alsoSubscribed = np.nan
            thisResult = pd.DataFrame({'collectionId' : int(i),
                                      'podcastSummary' : [podcastSummary],
                                      'episodeNames' : [episodeNames],
                                      'episodeDescriptions' : [episodeDescriptions],
                                      'alsoSubscribed' : [alsoSubscribed]})
            scrapeResults = pd.concat([scrapeResults, thisResult], axis=0)
            continue
    
    # get episode names
    episodeData = soup.findAll('button', kind='episode')
    try:
        episodeNames = [unidecode.unidecode(e['item-name']) for e in episodeData]
    except: # no name
        episodeNames = np.nan
    
    # get episode descriptions
    try:
        episodeDescriptions = [clean_description(e['description']) for e in episodeData]
    except: # no description
        episodeDescriptions = np.nan
    
    # get also subscribed podcasts
    alsoSubscribed = re.findall(r'adam-id="(\d+)" aria-label=', t.contents)
    try:
        alsoSubscribed = [int(x) for x in alsoSubscribed]
    except:
        alsoSubscribed = np.nan
        
    # append results
    thisResult = pd.DataFrame({'collectionId' : int(i),
                                      'podcastSummary' : [podcastSummary],
                                      'episodeNames' : [episodeNames],
                                      'episodeDescriptions' : [episodeDescriptions],
                                      'alsoSubscribed' : [alsoSubscribed]})
    scrapeResults = pd.concat([scrapeResults, thisResult], axis=0)

In [106]:
# save the results
scrapeResults['episodeDescriptions'] = [unicode(x) for x in scrapeResults['episodeDescriptions']]
scrapeResults['episodeNames'] = [unicode(x) for x in scrapeResults['episodeNames']]
scrapeResults['podcastSummary'] = [unicode(x) for x in scrapeResults['podcastSummary']]
scrapeResults.to_pickle('pkl/itunes_charts_scrape_' + time.strftime("%d-%m-%Y") + '.pkl')

In [135]:
# Construct iTunes API url
id_list = scrapeResults['collectionId'].tolist()
id_list = [int(x) for x in id_list]

api_base_url = 'https://itunes.apple.com/lookup?id=' 

In [157]:
colNames = [u'feedUrl',
 u'contentAdvisoryRating',
 u'trackRentalPrice',
 u'collectionExplicitness',
 u'releaseDate',
 u'currency',
 u'artistId',
 u'trackPrice',
 u'trackViewUrl',
 u'genres',
 u'collectionName',
 u'collectionId',
 u'trackId',
 u'artworkUrl600',
 u'collectionViewUrl',
 u'trackCount',
 u'primaryGenreName',
 u'collectionPrice',
 u'trackCensoredName',
 u'genreIds',
 u'trackName',
 u'artistViewUrl',
 u'kind',
 u'collectionHdPrice',
 u'trackHdRentalPrice',
 u'wrapperType',
 u'artworkUrl100',
 u'collectionCensoredName',
 u'trackHdPrice',
 u'radioStationUrl',
 u'artistName',
 u'artworkUrl60',
 u'trackExplicitness',
 u'artworkUrl30',
 u'country']

In [152]:
api_results = pd.DataFrame(columns=colNames)

for i in tqdm.tqdm(id_list):
    api_url = api_base_url + str(i)
    r = requests.get(api_url)
    text = r.text
    data = json.loads(text)
    thisResult = pd.io.json.json_normalize(data['results'])
    api_results = pd.concat([api_results, thisResult], axis=0)

In [163]:
# merge scrape & api results
podcastDf = pd.merge(api_results, scrapeResults, how = 'inner', on = 'collectionId')

In [166]:
# convert to int
podcastDf['collectionId'] = [int(x) for x in podcastDf['collectionId']]

In [169]:
# guess language of summary
podcastDf['language'] = [guess_language.guessLanguageName(x) for x in podcastDf['podcastSummary']]
podcastDf = podcastDf[podcastDf['language'] == 'English']

In [171]:
def clean_text(text):
    # remove mixed alphanumeric
    text = re.sub(r"""(?x) # verbose regex
                            \b    # Start of word
                            (?=   # Look ahead to ensure that this word contains...
                             \w*  # (after any number of alphanumeric characters)
                             \d   # ...at least one digit.
                            )     # End of lookahead
                            \w+   # Match the alphanumeric word
                            \s*   # Match any following whitespace""", 
                             "", text)
    
    # remove urls
    text = re.sub(r'\s([\S]*.com[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.org[\S]*)\b', '', text)

    
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    
    
    return text

In [172]:
# clean episode data
episodeDf = podcastDf[['collectionId','episodeDescriptions', 'episodeNames']]
episodeDf['episodeDescriptions'] = [x.split("\',") for x in episodeDf['episodeDescriptions']]
episodeDf['episodeNames'] = [x.split("\',") for x in episodeDf['episodeNames']]

clean_episode_description = []
for pod in episodeDf['episodeDescriptions']:
    clean_list = []
    for ep in pod:
        clean_list.append(clean_text(ep))
    clean_episode_description.append(clean_list)


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [174]:
episodeDf['clean_episode_description'] = clean_episode_description

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [175]:
clean_episode_name = []
for pod in episodeDf['episodeNames']:
    clean_list = []
    for ep in pod:
        clean_list.append(clean_text(ep))
    clean_episode_name.append(clean_list)
episodeDf['clean_episode_name'] = clean_episode_name

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [177]:
del episodeDf['episodeDescriptions']
del episodeDf['episodeNames']

In [178]:
podcastDf = pd.merge(podcastDf, episodeDf, how = 'inner', on='collectionId')

In [179]:
# clean podcast summary
podcastDf['podcastSummary'] = [clean_text(x) for x in podcastDf['podcastSummary']]

In [180]:
# save clean data
podcastDf.to_pickle('pkl/clean_podcast_data' + time.strftime("%d-%m-%Y") + '.pkl')

In [182]:
# connect to database
dbname = 'podcast'
username = 'lindsay'
con = psycopg2.connect(database = dbname, user = username)
cursor = con.cursor()

In [194]:
# upload data to database
query_id = "SELECT id FROM podcast WHERE collection_id = '%s';"
query_update = "INSERT INTO podcast (artwork_url30, artwork_url60, artwork_url100, artwork_url600, explicit, name, view_url, summary, episode_descriptions, episode_names, collection_id) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING id;"

for ind, thisPod in tqdm.tqdm(podcastDf.iterrows(), total=podcastDf.shape[0]):
    
    # check if in database
    cursor.execute(query_id, (thisPod['collectionId'], ))
    result = cursor.fetchall()
    
    if len(result) == 0:
        data = (thisPod['artworkUrl30'], thisPod['artworkUrl60'], thisPod['artworkUrl100'], thisPod['artworkUrl600'], thisPod['contentAdvisoryRating'], thisPod['collectionCensoredName'], thisPod['collectionViewUrl'], thisPod['podcastSummary'], thisPod['clean_episode_description'], thisPod['clean_episode_name'], thisPod['collectionId'])
        cursor.execute(query_update, data)

con.commit()

In [196]:
# get back ids
query_id = "SELECT id FROM podcast WHERE collection_id = '%s';"
db_ids = []
for ind, thisPod in podcastDf.iterrows():
    cursor.execute(query_id, (thisPod['collectionId'], ))
    db_ids.append(cursor.fetchall())
    
con.commit()

In [198]:
db_ids = [x[0] for x in db_ids]

In [202]:
db_ids = [x[0] for x in db_ids]

In [203]:
podcastDf['podcast_id'] = db_ids

In [206]:
# get unique artists
artistDf = podcastDf[['artistName', 'artistViewUrl']]
artistDf = artistDf.drop_duplicates()
artistDf.shape

(307, 2)

In [233]:
# insert artist data into artist table
query_artist = "SELECT id FROM artist WHERE name = %s;"
query_insert = "INSERT INTO artist (view_url, name) VALUES (%s, %s) RETURNING id;"
artist_id = []
for ind, row in artistDf.iterrows():
    data = (row['artistName'], )
    cursor.execute(query_artist, data)
    result = cursor.fetchone()[0]
    
    if result:
        artist_id.append(result)
    else:
        data = (row['artistViewUrl'], row['artistName'])
        cursor.execute(query_insert, data)
        artist_id.append(cursor.fetchone()[0])
        
con.commit()
artistDf['artist_id'] = artist_id



In [213]:
# get unique genres
genreDf = podcastDf['genres']
genreList = []
for row in genreDf:
    for genre in row:
        genreList.append(genre)
genreList = list(set(genreList))

In [241]:
# insert genre data into genre table
query_genre = "SELECT * FROM genre WHERE name = %s;"
query_insert = "INSERT INTO genre (name) VALUES (%s) RETURNING id;"
genre_id = []
for item in genreList:
    data = (item, )
    cursor.execute(query_genre, data)
    result = cursor.fetchone()[0]
    if result:
        genre_id.append(result)
        
    else:
        cursor.execute(query_insert, data)
        genre_id.append(cursor.fetchone()[0])
con.commit()
genreDf = pd.DataFrame({'name' : genreList,
                       'genre_id': genre_id})

In [242]:
podcast_artist = pd.merge(podcastDf, artistDf, how = 'inner', on = 'artistName')

In [243]:
# insert artist id into podcast table
query = "UPDATE podcast SET artist_id=(%s) WHERE id = (%s);"
for ind, row in podcast_artist.iterrows():
    data = (row['artist_id'], row['podcast_id'])
    cursor.execute(query, data)
    con.commit()

In [244]:
podcast_genre = pd.DataFrame(columns=['podcast_id', 'genre_name'])
for ind, row in podcastDf.iterrows():
    for genre in row['genres']:
        
        podcast_genre = podcast_genre.append(pd.DataFrame({'podcast_id' : [row['podcast_id']],
                                            'genre_name' : [genre]}))

In [245]:
podcast_genre = pd.merge(podcast_genre, genreDf, how = 'inner', left_on='genre_name', right_on='name')

In [246]:
podcast_genre.head()

,genre_name,podcast_id,genre_id,name
0,Arts,18521,205,Arts
1,Arts,13989,205,Arts
2,Arts,18523,205,Arts
3,Arts,18538,205,Arts
4,Arts,18539,205,Arts


In [247]:
# insert podcast-genre mappings into podcast_has_genre table
query = "INSERT INTO podcast_has_genre (podcast_id, genre_id) VALUES (%s, %s);"
for ind, row in podcast_genre.iterrows():
    data = (row['podcast_id'], row['genre_id'])
    cursor.execute(query, data)
con.commit()

In [250]:
tmp_also_subscribed = podcastDf[['podcast_id', 'alsoSubscribed']]
also_subscribed = pd.DataFrame(columns = ['podcast_id', 'also_subscribed'])
for ind, row in tmp_also_subscribed.iterrows():
    for sub in row['alsoSubscribed']:
        also_subscribed = also_subscribed.append(pd.DataFrame({'podcast_id' : [row['podcast_id']],
                                                              'also_subscribed' : [sub]}))

In [252]:
also_subscribed.rename(columns = {'podcast_id':'initial_podcast_id'}, inplace=True)
podcastId = podcastDf[['podcast_id', 'collectionId']]


In [254]:
also_subscribed = pd.merge(also_subscribed, podcastId, how = 'inner', left_on='also_subscribed', right_on='collectionId')

In [255]:
# insert also_subscribed data
query = "INSERT INTO also_subscribed (initial_podcast, subscribed_podcast) VALUES (%s, %s);"
for ind, row in also_subscribed.iterrows():
    data = (row['initial_podcast_id'], row['podcast_id'])
    cursor.execute(query, data)
con.commit()

In [256]:
# pkl dfs
podcastDf.to_pickle('pkl/podcastDf_with_sql_id' + time.strftime("%d-%m-%Y") + '.pkl')
artistDf.to_pickle('pkl/artistDf_with_sql_id.pkl' + time.strftime("%d-%m-%Y") + '.pkl')
genreDf.to_pickle('pkl/genreDf_with_sql_id.pkl' + time.strftime("%d-%m-%Y") + '.pkl')

In [316]:
query = "SELECT id, collection_id, summary, episode_descriptions, episode_names FROM podcast"
cursor.execute(query, con)
query_results = cursor.fetchall()

In [320]:
query_df = pd.DataFrame({'id' : [x[0] for x in query_results],
                         'collection_id' : [x[1] for x in query_results],
                         'summary' : [x[2] for x in query_results],
                         'episode_descriptions' : [x[3] for x in query_results],
                         'episode_names' : [x[4] for x in query_results]})

In [324]:
# concatenate text and remove non-alphanumeric characters for each podcast
podcast_text = pd.DataFrame(columns=['id', 'collection_id', 'text', 'language'])
for ind, row in query_df.iterrows():
    # concatenate
    text = ' '.join([row['summary'], row['episode_descriptions'], row['episode_names']])
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    # guess language of text
    language = guess_language.guessLanguageName(text)
    
    podcast_text = podcast_text.append(pd.DataFrame({'id' : [row['id']],
                                                     'collection_id': [row['collection_id']],
                                                    'text' : [text],
                                                    'language' : [language]}))

In [325]:
podcast_text.head()

,collection_id,id,language,text
0,126723118,12350,English,Insurance news interviews rating announcements...
0,699748055,12351,English,The Amovetv crew talk video games eSports a lo...
0,258723953,18521,English,A weekly conversation about whats new in The N...
0,529175048,12357,English,AfricanAmerican Conservatives focuses on topic...
0,290783428,18527,English,Money makes the world go around faster and fas...


In [326]:
# create tokenizer
tokenizer = RegexpTokenizer(r'\w+')

# tokenize text
tokenized = podcast_text.copy(deep=True)
tokenized['text'] = [tokenizer.tokenize(x.lower()) for x in tokenized['text']]

# create list of stop words
stop = get_stop_words('en')

# remove non-alphanumeric, non-space
stop = [re.sub(r'([^\s\w]|_)+', '', x) for x in stop]

# add in custom stop words
days = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
other = ['nan', 'podcast']

[stop.append(unicode(day)) for day in days]
[stop.append(unicode(month)) for month in months]
[stop.append(unicode(x)) for x in other]

[None, None]

In [327]:
def remove_stop(text, stop):
    new_text = []
    for word in text:
        if word not in stop:
            new_text.append(word)
    return new_text

In [328]:
# remove stop words
stopped = tokenized.copy(deep=True)
stopped['text'] = [remove_stop(text,stop) for text in stopped['text']]

In [329]:
def stem_list(text, p_stemmer):
    new_list = []
    for word in text:
        new_list.append(p_stemmer.stem(word))
    return new_list

In [330]:
# stem words
p_stemmer = PorterStemmer()
stemmed = stopped.copy(deep=True)
stemmed['text'] = [stem_list(text, p_stemmer) for text in stemmed['text']]

In [331]:
#remove tokens containing 'http' or 'www'
documents_no_web = pd.DataFrame(columns=stemmed.columns)
for ind, row in stemmed.iterrows():
    text = row['text']
    new_text = []
    for word in text:
        if re.search(r'http', word):
            continue
        if re.search(r'www', word):
            continue
        new_text.append(word)
    row['text'] = new_text
    documents_no_web = documents_no_web.append(row)

In [332]:
documents_no_web.head()

,collection_id,id,language,text
0,126723118,12350,English,"[insur, news, interview, rate, announc, insur,..."
0,699748055,12351,English,"[amovetv, crew, talk, video, game, esport, lot..."
0,258723953,18521,English,"[weekli, convers, new, new, yorker, orson, wel..."
0,529175048,12357,English,"[africanamerican, conserv, focus, topic, relev..."
0,290783428,18527,English,"[money, make, world, go, around, faster, faste..."


In [333]:
documents_no_web.shape

(6136, 4)

In [335]:
dictionary = corpora.Dictionary.load('flask_app/flask_podcast/static/data/dictionary.dict')
lsi = models.LsiModel.load('flask_app/flask_podcast/static/data/model.lsi')
index = similarities.MatrixSimilarity.load('flask_app/flask_podcast/static/data/tfidf_lsi_similarities.index')

In [336]:
corpus = [dictionary.doc2bow(text) for text in documents_no_web['text']]

In [337]:
# transform to tfidf
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpora.MmCorpus.serialize('flask_app/flask_podcast/static/data/corpus_tfidf.mm', corpus_tfidf)

In [338]:
# lsi model on tfidf
%time lsi = models.lsimodel.LsiModel(corpus_tfidf, num_topics = 100, id2word=dictionary)
lsi.save('flask_app/flask_podcast/static/data/model.lsi')

CPU times: user 28.5 s, sys: 3.63 s, total: 32.1 s
Wall time: 26.2 s


In [339]:
# calculate similarities
index = similarities.MatrixSimilarity(lsi[corpus_tfidf])
index.save('flask_app/flask_podcast/static/data/tfidf_lsi_similarities.index')

In [340]:
new_ids = pd.DataFrame({'podcast_id': podcastDf.podcast_id})
new_ids.head()

,podcast_id
0,18521
1,18522
2,13989
3,18523
4,18524


In [361]:
new_ids.to_pickle('flask_app/flask_podcast/static/data/podcast_id_to_gensim_id.pkl')

In [362]:
new_ids.shape

(396, 1)

In [346]:
new_podcasts = pd.read_pickle('pkl/itunes_charts_scrape_07-02-2016.pkl')

In [347]:
new_podcasts.columns

Index([u'alsoSubscribed', u'collectionId', u'episodeDescriptions',
       u'episodeNames', u'podcastSummary'],
      dtype='object')

In [348]:
podcastDf.columns

Index([                 u'artistId',                u'artistName',
                   u'artistViewUrl',             u'artworkUrl100',
                    u'artworkUrl30',              u'artworkUrl60',
                   u'artworkUrl600',    u'collectionCensoredName',
          u'collectionExplicitness',         u'collectionHdPrice',
                    u'collectionId',            u'collectionName',
                 u'collectionPrice',         u'collectionViewUrl',
           u'contentAdvisoryRating',                   u'country',
                        u'currency',                   u'feedUrl',
                        u'genreIds',                    u'genres',
                            u'kind',          u'primaryGenreName',
                 u'radioStationUrl',               u'releaseDate',
               u'trackCensoredName',                u'trackCount',
               u'trackExplicitness',              u'trackHdPrice',
              u'trackHdRentalPrice',                   u'track

In [349]:
new_podcasts = pd.merge(new_podcasts, podcastDf, how='inner', on='collectionId')

In [350]:
new_podcasts.head()

,alsoSubscribed_x,collectionId,episodeDescriptions_x,episodeNames_x,podcastSummary_x,artistId,artistName,artistViewUrl,artworkUrl100,artworkUrl30,...,trackViewUrl,wrapperType,alsoSubscribed_y,episodeDescriptions_y,episodeNames_y,podcastSummary_y,language,clean_episode_description,clean_episode_name,podcast_id
0,"[120315179, 279188498, 158004641, 73330715, 21...",258723953,"['Orson Welles was born a hundred years ago, i...","['Beyond ""Citizen Kane""', 'What it Means to Be...",A weekly conversation about what's new in The ...,331376890,The New Yorker,https://itunes.apple.com/us/artist/the-new-yor...,http://is5.mzstatic.com/image/thumb/Music1/v4/...,http://is5.mzstatic.com/image/thumb/Music1/v4/...,...,https://itunes.apple.com/us/podcast/new-yorker...,track,"[120315179, 279188498, 158004641, 73330715, 21...","['Orson Welles was born a hundred years ago, i...","['Beyond ""Citizen Kane""', 'What it Means to Be...",A weekly conversation about whats new in The N...,English,[Orson Welles was born a hundred years ago in ...,"[Beyond Citizen Kane, What it Means to Be a S...",18521
1,[],1076128054,"[""Join hosts Adam Hlavac, Hector Navarro and A...","['45: Is Apokolips Coming?', '44: Justice Leag...",Welcome to Superhero News! Your source for the...,NaN,Superhero News,NaN,http://is4.mzstatic.com/image/thumb/Music69/v4...,http://is4.mzstatic.com/image/thumb/Music69/v4...,...,https://itunes.apple.com/us/podcast/the-superh...,track,[],"[""Join hosts Adam Hlavac, Hector Navarro and A...","['45: Is Apokolips Coming?', '44: Justice Leag...",Welcome to Superhero News Your source for the ...,English,[Join hosts Adam Hlavac Hector Navarro and Agu...,"[ Is Apokolips Coming, Justice League Concep...",18522
2,"[84389707, 809264944, 941907967, 490297492, 95...",870688022,"['At 63 years old, musician Lucinda Williams i...",['Lucinda Williams Says Whatever the Hell She ...,"Death, Sex &amp; Money is a podcast about the ...",127981066,WNYC Studios,https://itunes.apple.com/us/artist/wnyc/id1279...,http://is4.mzstatic.com/image/thumb/Music2/v4/...,http://is4.mzstatic.com/image/thumb/Music2/v4/...,...,https://itunes.apple.com/us/podcast/death-sex-...,track,"[84389707, 809264944, 941907967, 490297492, 95...","['At 63 years old, musician Lucinda Williams i...",['Lucinda Williams Says Whatever the Hell She ...,Death Sex amp Money is a podcast about the big...,English,[At years old musician Lucinda Williams is mor...,[Lucinda Williams Says Whatever the Hell She W...,13989
3,"[953290300, 275699983, 394775318, 523121474, 2...",283657561,"['All-NEW ""Snap LIVE! in NYC"" featuring Tariq ...","['Snap #703 - Snap LIVE! in NYC', 'Snap #607 -...","Snap Judgment (Storytelling, with a BEAT) mixe...",127981066,Snap Judgment and WNYC Studios,https://itunes.apple.com/us/artist/wnyc/id1279...,http://is5.mzstatic.com/image/thumb/Music6/v4/...,http://is5.mzstatic.com/image/thumb/Music6/v4/...,...,https://itunes.apple.com/us/podcast/snap-judgm...,track,"[953290300, 275699983, 394775318, 523121474, 2...","['All-NEW ""Snap LIVE! in NYC"" featuring Tariq ...","['Snap #703 - Snap LIVE! in NYC', 'Snap #607 -...",Snap Judgment Storytelling with a BEAT mixes r...,English,[AllNEW Snap LIVE in NYC featuring Tariq Black...,"[Snap Snap LIVE in NYC, Snap infamous, Sna...",18523
4,"[492735082, 278196007, 74840240, 81210923, 789...",336934080,"['FRONTLINE, The New York Times and the Canadi...","['Supplements and Safety', 'Netanyahu at War',...",FRONTLINE presents audio versions of select fu...,127966016,FRONTLINE,https://itunes.apple.com/us/artist/pbs/id12796...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,http://is1.mzstatic.com/image/thumb/Music6/v4/...,...,https://itunes.apple.com/us/podcast/frontline-...,track,"[492735082, 278196007, 74840240, 81210923, 789...","['FRONTLINE, The New York Times and the Canadi...","['Supplements and Safety', 'Netanyahu at War',...",FRONTLINE presents audio versions of select fu...,English,[FRONTLINE The New York Times and the Canadian...,"[Supplemen

In [351]:
query = """
UPDATE podcast
SET raw_summary = (%s)
WHERE id = (%s);
"""
query = query.replace('\n', ' ')

In [355]:
for ind, row in tqdm.tqdm(new_podcasts.iterrows(), total=new_podcasts.shape[0]):
    data = ((row['podcastSummary_x'], ), (row['podcast_id'], ))
    cursor.execute(query, data)
    
con.commit()

In [358]:
query = "SELECT clean_name, id FROM podcast;"
matching_results = pd.read_sql_query(query, con)
matching_results['clean_name'] = [x.decode('utf-8') for x in matching_results['clean_name']]
matching_results['clean_name'] = [x.replace('"', "'") for x in matching_results['clean_name']]
match_json = matching_results.to_json(None,"records")

In [368]:
query = "SELECT id, collection_id FROM podcast;"
cursor.execute(query)
results = cursor.fetchall()

In [369]:
len(results)

6136

In [370]:
id_mapping = pd.DataFrame({'podcast_id' : [x[0] for x in results],
                           'collection_id' : [x[1] for x in results]})

In [371]:
id_mapping

,collection_id,podcast_id
0,258723953,18521
1,1076128054,18522
2,126723118,12350
3,699748055,12351
4,775174336,18373
5,870688022,13989
6,283657561,18523
7,276268226,18525
8,529175048,12357
9,336934080,18524


In [372]:
documents_no_web.shape

(6136, 4)

In [374]:
id_mapping = pd.DataFrame({'podcast_id' : documents_no_web['id'].tolist()})

In [376]:
id_mapping.to_pickle('flask_app/flask_podcast/static/data/podcast_id_to_gensim_id.pkl')